In [19]:
import sys
import logging

import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
from lrann.datasets import Loader, random_train_test_split
from lrann.estimators import ImplicitEst
from lrann.models import BilinearNet, DeepNet
from lrann.evaluation import mrr_score, precision_recall_score
from lrann.utils import is_cuda_available

In [64]:
data = Loader().load_movielens('100k-old')

In [65]:
train, test = random_train_test_split(data)

## Estimator using a low-rank approximation model

In [82]:
lra_model = BilinearNet(data.n_users, data.n_items, embedding_dim=32, sparse=False)
lra_est = ImplicitEst(model=lra_model, 
                      n_iter=5, 
                      use_cuda=is_cuda_available())
lra_est.fit(train, verbose=True)

Epoch 0: loss 0.2794700464248657
Epoch 1: loss 0.173495687687397
Epoch 2: loss 0.15406905018091202
Epoch 3: loss 0.14468476692438126
Epoch 4: loss 0.1399620051383972
Epoch 5: loss 0.13484953826069832
Epoch 6: loss 0.1303108807146549
Epoch 7: loss 0.12643663268089295
Epoch 8: loss 0.1252969019472599
Epoch 9: loss 0.12476854658722877


In [83]:
prec, recall = precision_recall_score(lra_est, test)
prec.mean(), recall.mean()

(0.1169851380042463, 0.09109336034123514)

In [84]:
mrr_score(lra_est, train).mean(), mrr_score(lra_est, test).mean()

(0.053140688285031636, 0.04385634987488505)

## Estimator using a deep neural networks model

In [85]:
nn_model = DeepNet(data.n_users, data.n_items, embedding_dim=32, sparse=False)

In [ ]:
nn_est = ImplicitEst(model=nn_model, 
                     n_iter=5, 
                     use_cuda=is_cuda_available())
nn_est.fit(train, verbose=True)

Epoch 0: loss 0.23867404057979583
Epoch 1: loss 0.21159768166542053
Epoch 2: loss 0.21070225497484207
Epoch 3: loss 0.207650233066082


In [87]:
prec, recall = precision_recall_score(nn_est, test)
prec.mean(), recall.mean()

(0.11634819532908704, 0.09394370589661069)

In [81]:
mrr_score(nn_est, train).mean(), mrr_score(nn_est, test).mean()

(0.051961728520640696, 0.040767875373158284)